# Assignment 3

In [1]:
# Imports
import pandas as pd
import os
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

### Connect to the Neo4j server and to use Neo4j GDS

In [2]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [3]:
NEO4J_AUTH = ( "neo4j",  "12072003") ### Make sure the password is yours

In [4]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [11]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

### Step 5 Create the SHARED_PII relationship (between clients) and the resulting subgraph

In [12]:
# MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(pii)<-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
# WHERE c1 <> c2
# MERGE (c1)-[:SHARED_PII]->(c2)

### Step 6 Cypher Query to return clientId and clusterId

In [13]:
# CALL gds.wcc.stream('clientClusters')
# YIELD nodeId, componentId
# RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId
